# Estimate domains of translated protein product of isoforms

This notebook requires that the `hmmscan` compute job in [0.2.11] be complete.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
# splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(folder), index_col=0)
# print splicing_feature_data.shape
# splicing_feature_data.head()

In [2]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)


splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

In [4]:
! ls $alternative_feature_folder/*translation*

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa_iupred_out
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa_iupred_tmp
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt.err
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt.out
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/protein_translations.csv


In [5]:
! ln -s $alternative_feature_folder/protein_translations.csv $alternative_feature_folder/isoform_translations.csv

In [6]:
isoform_translations = pd.read_csv('{}/isoform_translations.csv'.format(alternative_feature_folder), index_col=0)
isoform_translations.head()

,isoform1_translation,isoform2_translation
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSA...
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,NaN,ISFSYLCPASWYFTVPTVSPFLRQRVAFLGLFFISCLLLLMLIIDF...
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,NaN,LPETHQLLQLRRFAHGTAGLVFLTALSGAFVAGLDAGLVYNSFPKM...
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQEVV,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQAK...


In [8]:
prefix = 'isoform_translations'
translated_fasta = '{}/{}.fa'.format(alternative_feature_folder, prefix)

### Read HMMScan output

In [9]:
pd.options.display.max_columns = 30

In [10]:
from poshsplice.hmmscan import read_hmmscan

hmmscan_out = '{}/{}_hmmscan_pfamA.txt'.format(alternative_feature_folder, prefix)
! ls -lha $hmmscan_out
hmmscan = read_hmmscan(hmmscan_out)
print hmmscan.shape
hmmscan.head()

-rw-r--r-- 1 obotvinnik yeo-group 28M Apr 11 20:34 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt
(74296, 23)


/home/obotvinnik/workspace-git/poshsplice/poshsplice/hmmscan.py:35: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)


,target_name,target_accession,target_length,query_name,query_accession,query_length,sequence_e_value,sequence_score,sequence_bias,domain_number,domain_total,domain_conditional_e_value,domain_independent_e_value,domain_score,domain_bias,target_start,target_stop,query_start,query_stop,query_domain_envelope_start,query_domain_envelope_stop,mean_posterior_probability,target_description
0,Aminotran_1_2,PF00155.16,363,exon:chr10:101165513-101165617:-@exon:chr10:10...,-,140,1.600000e-37,129.3,0.0,1,1,1.200000e-41,1.700000e-37,129.2,0.0,142,270,2,138,1,140,0.96,Aminotransferase class I and II
1,GDA1_CD39,PF01150.12,436,exon:chr10:101421203-101421385:+@exon:chr10:10...,-,179,1.800000e-27,95.9,0.0,1,1,2.800000e-31,2.100000e-27,95.7,0.0,4,104,75,179,72,179,0.90,GDA1/CD39 (nucleoside phosphatase) family
2,OppC_N,PF12911.2,56,exon:chr10:101421203-101421385:+@exon:chr10:10...,-,179,8.500000e-02,12.2,0.2,1,1,2.200000e-05,1.600000e-01,11.3,0.2,12,37,21,46,17,56,0.92,N-terminal TM domain of oligopeptide transport...
3,COX15-CtaA,PF02628.10,302,exon:chr10:101480744-101480825:-@exon:chr10:10...,-,117,8.100000e-23,80.7,0.2,1,1,6.300000e-27,9.300000e-23,80.5,0.2,165,270,7,117,2,117,0.85,Cytochrome oxidase assembly protein
4,CutC,PF03932.9,202,exon:chr10:101507014-101507147:+@exon:chr10:10...,-,88,8.500000e-24,83.9,0.0,1,1,6.400000e-28,9.500000e-24,83.7,0.0,123,201,1,79,1,80,0.99,CutC family


Split query name (event name + cds + isoform information) into separate columns

In [11]:
hmmscan.query_name.head().values

array([ 'exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-|CDS:chr10:101165513-101165617:-:0@CDS:chr10:101163481-101163631:-:0@CDS:chr10:101163226-101163391:-:2|isoform2',
       'exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+|CDS:chr10:101421203-101421385:+:1@CDS:chr10:101439018-101439223:+:1@CDS:chr10:101439482-101439632:+:2|isoform2',
       'exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+|CDS:chr10:101421203-101421385:+:1@CDS:chr10:101439018-101439223:+:1@CDS:chr10:101439482-101439632:+:2|isoform2',
       'exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-|CDS:chr10:101480744-101480825:-:0@CDS:chr10:101478103-101478257:-:2@CDS:chr10:101476105-101476218:-:0|isoform2',
       'exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+|CDS:chr10:10

In [12]:
split_ids = hmmscan.query_name.str.split('|').apply(pd.Series)
split_ids.head()

,0,1,2
0,exon:chr10:101165513-101165617:-@exon:chr10:10...,CDS:chr10:101165513-101165617:-:0@CDS:chr10:10...,isoform2
1,exon:chr10:101421203-101421385:+@exon:chr10:10...,CDS:chr10:101421203-101421385:+:1@CDS:chr10:10...,isoform2
2,exon:chr10:101421203-101421385:+@exon:chr10:10...,CDS:chr10:101421203-101421385:+:1@CDS:chr10:10...,isoform2
3,exon:chr10:101480744-101480825:-@exon:chr10:10...,CDS:chr10:101480744-101480825:-:0@CDS:chr10:10...,isoform2
4,exon:chr10:101507014-101507147:+@exon:chr10:10...,CDS:chr10:101507014-101507147:+:2@CDS:chr10:10...,isoform2


In [13]:
split_ids = split_ids.rename(columns={0: 'event_name', 1:'cds', 2: 'isoform'})
split_ids.head()

,event_name,cds,isoform
0,exon:chr10:101165513-101165617:-@exon:chr10:10...,CDS:chr10:101165513-101165617:-:0@CDS:chr10:10...,isoform2
1,exon:chr10:101421203-101421385:+@exon:chr10:10...,CDS:chr10:101421203-101421385:+:1@CDS:chr10:10...,isoform2
2,exon:chr10:101421203-101421385:+@exon:chr10:10...,CDS:chr10:101421203-101421385:+:1@CDS:chr10:10...,isoform2
3,exon:chr10:101480744-101480825:-@exon:chr10:10...,CDS:chr10:101480744-101480825:-:0@CDS:chr10:10...,isoform2
4,exon:chr10:101507014-101507147:+@exon:chr10:10...,CDS:chr10:101507014-101507147:+:2@CDS:chr10:10...,isoform2


In [14]:
split_ids.shape

(74296, 3)

In [15]:
hmmscan.shape

(74296, 23)

In [16]:
hmmscan_ids = pd.concat([hmmscan, split_ids], axis=1)
hmmscan_ids.head()

,target_name,target_accession,target_length,query_name,query_accession,query_length,sequence_e_value,sequence_score,sequence_bias,domain_number,domain_total,domain_conditional_e_value,domain_independent_e_value,domain_score,domain_bias,target_start,target_stop,query_start,query_stop,query_domain_envelope_start,query_domain_envelope_stop,mean_posterior_probability,target_description,event_name,cds,isoform
0,Aminotran_1_2,PF00155.16,363,exon:chr10:101165513-101165617:-@exon:chr10:10...,-,140,1.600000e-37,129.3,0.0,1,1,1.200000e-41,1.700000e-37,129.2,0.0,142,270,2,138,1,140,0.96,Aminotransferase class I and II,exon:chr10:101165513-101165617:-@exon:chr10:10...,CDS:chr10:101165513-101165617:-:0@CDS:chr10:10...,isoform2
1,GDA1_CD39,PF01150.12,436,exon:chr10:101421203-101421385:+@exon:chr10:10...,-,179,1.800000e-27,95.9,0.0,1,1,2.800000e-31,2.100000e-27,95.7,0.0,4,104,75,179,72,179,0.90,GDA1/CD39 (nucleoside phosphatase) family,exon:chr10:101421203-101421385:+@exon:chr10:10...,CDS:chr10:101421203-101421385:+:1@CDS:chr10:10...,isoform2
2,OppC_N,PF12911.2,56,exon:chr10:101421203-101421385:+@exon:chr10:10...,-,179,8.500000e-02,12.2,0.2,1,1,2.200000e-05,1.600000e-01,11.3,0.2,12,37,21,46,17,56,0.92,N-terminal TM domain of oligopeptide transport...,exon:chr10:101421203-101421385:+@exon:chr10:10...,CDS:chr10:101421203-101421385:+:1@CDS:chr10:10...,isoform2
3,COX15-CtaA,PF02628.10,302,exon:chr10:101480744-101480825:-@exon:chr10:10...,-,117,8.100000e-23,80.7,0.2,1,1,6.300000e-27,9.300000e-23,80.5,0.2,165,270,7,117,2,117,0.85,Cytochrome oxidase assembly protein,exon:chr10:101480744-101480825:-@exon:chr10:10...,CDS:chr10:101480744-101480825:-:0@CDS:chr10:10...,isoform2
4,CutC,PF03932.9,202,exon:chr10:101507014-101507147:+@exon:chr10:10...,-,88,8.500000e-24,83.9,0.0,1,1,6.400000e-28,9.500000e-24,83.7,0.0,123,201,1,79,1,80,0.99,CutC family,exon:chr10:101507014-101507147:+@exon:chr10:10...,CDS:chr10:101507014-101507147:+:2@CDS:chr10:10...,isoform2


In [17]:
pd.options.display.max_colwidth = 100

In [18]:
e_value_cutoff = 1e-5
hmmscan_filtered = hmmscan_ids.loc[hmmscan_ids.domain_independent_e_value < e_value_cutoff]

### Add protein translations

To check if the isoforms went from no annotated reading frame to either a translation OR domain.

In [21]:
translations_tidy = isoform_translations.unstack().reset_index()
translations_tidy = translations_tidy.rename(columns={'level_0': 'isoform', 'level_1': 'event_name', 0: 'translation'})
translations_tidy['isoform'] = translations_tidy['isoform'].str.split('_').str[0]
translations_tidy.head()

,isoform,event_name,translation
0,isoform1,exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN
1,isoform1,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN
2,isoform1,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,NaN
3,isoform1,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,NaN
4,isoform1,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQEVV


In [22]:
merge_cols = ['isoform', 'event_name']
print hmmscan_filtered.shape
hmmscan_filtered = hmmscan_filtered.merge(translations_tidy, how='outer', left_on=merge_cols, right_on=merge_cols)
print hmmscan_filtered.shape
hmmscan_filtered.head()

(15746, 26)
(48730, 27)


,target_name,target_accession,target_length,query_name,query_accession,query_length,sequence_e_value,sequence_score,sequence_bias,domain_number,domain_total,domain_conditional_e_value,domain_independent_e_value,domain_score,domain_bias,target_start,target_stop,query_start,query_stop,query_domain_envelope_start,query_domain_envelope_stop,mean_posterior_probability,target_description,event_name,cds,isoform,translation
0,Aminotran_1_2,PF00155.16,363,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391...,-,140,1.600000e-37,129.3,0.0,1,1,1.200000e-41,1.700000e-37,129.2,0.0,142,270,2,138,1,140,0.96,Aminotransferase class I and II,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,CDS:chr10:101165513-101165617:-:0@CDS:chr10:101163481-101163631:-:0@CDS:chr10:101163226-10116339...,isoform2,NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSAYQGFASGNLERDAWAIRYFVSEGFEFFCAQSFSKNFGLYNERVGNLTVVG...
1,GDA1_CD39,PF01150.12,436,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632...,-,179,1.800000e-27,95.9,0.0,1,1,2.800000e-31,2.100000e-27,95.7,0.0,4,104,75,179,72,179,0.90,GDA1/CD39 (nucleoside phosphatase) family,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,CDS:chr10:101421203-101421385:+:1@CDS:chr10:101439018-101439223:+:1@CDS:chr10:101439482-10143963...,isoform2,ISFSYLCPASWYFTVPTVSPFLRQRVAFLGLFFISCLLLLMLIIDFRHWSASLPRDRQYERYLARVGELEATDTEDPNLNYGLVVDCGSSGSRIFV...
2,COX15-CtaA,PF02628.10,302,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218...,-,117,8.100000e-23,80.7,0.2,1,1,6.300000e-27,9.300000e-23,80.5,0.2,165,270,7,117,2,117,0.85,Cytochrome oxidase assembly protein,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,CDS:chr10:101480744-101480825:-:0@CDS:chr10:101478103-101478257:-:2@CDS:chr10:101476105-10147621...,isoform2,LPETHQLLQLRRFAHGTAGLVFLTALSGAFVAGLDAGLVYNSFPKMGESWIPEDLFTFSPILRNVFENPTMVQFDHRILGITSVTAITVLYFLSRR...
3,CutC,PF03932.9,202,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391...,-,88,8.500000e-24,83.9,0.0,1,1,6.400000e-28,9.500000e-24,83.7,0.0,123,201,1,79,1,80,0.99,CutC family,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,CDS:chr10:101507014-101507147:+:2@CDS:chr10:101510126-101510153:+:0@CDS:chr10:101514286-10151439...,isoform2,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQAKGRIVVMPGGGITDRNLQRILEGSGATEFHCSARSTRDSGMKF
4,CutC,PF03932.9,202,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391...,-,47,1.700000e-11,43.7,0.0,1,1,1.300000e-15,1.900000e-11,43.6,0.0,123,166,1,44,1,46,0.98,CutC family,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,CDS:chr10:101507014-101507147:+:2@CDS:chr10:101514286-101514294:+:0,isoform1,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQEVV


In [23]:
print hmmscan_filtered.shape
hmmscan_filtered = hmmscan_filtered.dropna(subset=['translation'], axis=0)
print hmmscan_filtered.shape

(48730, 27)
(30395, 27)


## Annotate with Pfam-A clans and Gene ontology

First add a column that has the PFam accession ID ("`target_accession`") without the period/decimal and everything after it.

In [24]:
hmmscan_filtered['pfam_accession_no_version'] = hmmscan_filtered.target_accession.str.split('.').str[0]

### Pfam-A clans

In [25]:
column_names = ['pfamA_accession', 'clan_accession', 'clan_id', 'pfamA_id', 'pfamA_description']
pfam_clans = pd.read_table('/projects/ps-yeolab/genomes/pfam/release_27/ftp.sanger.ac.uk/pub/databases/Pfam/releases/Pfam27.0/Pfam-A.clans.tsv',
                           names=column_names, index_col=0,
                           header=None)
pfam_clans = pfam_clans.replace('\\N', np.nan)
print pfam_clans.shape
pfam_clans.head()

(14831, 4)


,clan_accession,clan_id,pfamA_id,pfamA_description
pfamA_accession,,,,
PF00389,CL0325,Form_Glyc_dh,2-Hacid_dh,"D-isomer specific 2-hydroxyacid dehydrogenase, catalytic domain"
PF00198,CL0149,CoA-acyltrans,2-oxoacid_dh,2-oxoacid dehydrogenases acyltransferase (catalytic domain)
PF04029,NaN,NaN,2-ph_phosp,2-phosphosulpholactate phosphatase
PF03171,CL0029,Cupin,2OG-FeII_Oxy,2OG-Fe(II) oxygenase superfamily
PF01073,CL0063,NADP_Rossmann,3Beta_HSD,3-beta hydroxysteroid dehydrogenase/isomerase family


Join to `hmmscan_filtered`

In [26]:
hmmscan_clans = hmmscan_filtered.join(pfam_clans, on='pfam_accession_no_version')
hmmscan_clans.head()

,target_name,target_accession,target_length,query_name,query_accession,query_length,sequence_e_value,sequence_score,sequence_bias,domain_number,domain_total,domain_conditional_e_value,domain_independent_e_value,domain_score,domain_bias,...,query_start,query_stop,query_domain_envelope_start,query_domain_envelope_stop,mean_posterior_probability,target_description,event_name,cds,isoform,translation,pfam_accession_no_version,clan_accession,clan_id,pfamA_id,pfamA_description
0,Aminotran_1_2,PF00155.16,363,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391...,-,140,1.600000e-37,129.3,0.0,1,1,1.200000e-41,1.700000e-37,129.2,0.0,...,2,138,1,140,0.96,Aminotransferase class I and II,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,CDS:chr10:101165513-101165617:-:0@CDS:chr10:101163481-101163631:-:0@CDS:chr10:101163226-10116339...,isoform2,NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSAYQGFASGNLERDAWAIRYFVSEGFEFFCAQSFSKNFGLYNERVGNLTVVG...,PF00155,CL0061,PLP_aminotran,Aminotran_1_2,Aminotransferase class I and II
1,GDA1_CD39,PF01150.12,436,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632...,-,179,1.800000e-27,95.9,0.0,1,1,2.800000e-31,2.100000e-27,95.7,0.0,...,75,179,72,179,0.90,GDA1/CD39 (nucleoside phosphatase) family,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,CDS:chr10:101421203-101421385:+:1@CDS:chr10:101439018-101439223:+:1@CDS:chr10:101439482-10143963...,isoform2,ISFSYLCPASWYFTVPTVSPFLRQRVAFLGLFFISCLLLLMLIIDFRHWSASLPRDRQYERYLARVGELEATDTEDPNLNYGLVVDCGSSGSRIFV...,PF01150,CL0108,Actin_ATPase,GDA1_CD39,GDA1/CD39 (nucleoside phosphatase) family
2,COX15-CtaA,PF02628.10,302,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218...,-,117,8.100000e-23,80.7,0.2,1,1,6.300000e-27,9.300000e-23,80.5,0.2,...,7,117,2,117,0.85,Cytochrome oxidase assembly protein,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,CDS:chr10:101480744-101480825:-:0@CDS:chr10:101478103-101478257:-:2@CDS:chr10:101476105-10147621...,isoform2,LPETHQLLQLRRFAHGTAGLVFLTALSGAFVAGLDAGLVYNSFPKMGESWIPEDLFTFSPILRNVFENPTMVQFDHRILGITSVTAITVLYFLSRR...,PF02628,CL0328,2heme_cytochrom,COX15-CtaA,Cytochrome oxidase assembly protein
3,CutC,PF03932.9,202,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391...,-,88,8.500000e-24,83.9,0.0,1,1,6.400000e-28,9.500000e-24,83.7,0.0,...,1,79,1,80,0.99,CutC family,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,CDS:chr10:101507014-101507147:+:2@CDS:chr10:101510126-101510153:+:0@CDS:chr10:101514286-10151439...,isoform2,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQAKGRIVVMPGGGITDRNLQRILEGSGATEFHCSARSTRDSGMKF,PF03932,CL0036,TIM_barrel,CutC,CutC family
4,CutC,PF03932.9,202,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391...,-,47,1.700000e-11,43.7,0.0,1,1,1.300000e-15,1.900000e-11,43.6,0.0,...,1,44,1,46,0.98,CutC family,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,CDS:chr10:101507014-101507147:+:2@CDS:chr10:101514286-101514294:+:0,isoform1,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQEVV,PF03932,CL0036,TIM_barrel,CutC,CutC family


In [27]:
hmmscan_clans.shape

(30395, 32)

## Get the net effect of domain changes from isoform1 to isoform2

If the Pfam Clan of the domain switches from isoform1 to isoform2 --> Call this "switching domains"

In [28]:
hmmscan_clans_isoform = hmmscan_clans.set_index('isoform').sort_index()

In [29]:
hmmscan_clans_isoform.groupby('event_name')

In [30]:
def get_col_set(df, isoform, col):
    try:
        return set(df.loc[isoform][col].values)
    except AttributeError:
        # Only one item --> str or float has no attribute "values"
        return set([df.loc[isoform][col]])
    
def get_domain_effects(df):
    
    if 'isoform1' not in df.index:
        return 'No annotated reading frame --> Protein +/- domain'
    elif 'isoform2' not in df.index:
        return 'Protein +/- domain --> No annotated reading frame'
    
    
    df_clans = df.dropna(subset=['clan_id'], axis=0)
    try:
        isoform1 = get_col_set(df_clans, 'isoform1', 'clan_id')
        isoform2 = get_col_set(df_clans, 'isoform2', 'clan_id')
        
        # Get domains unique to either isoform
        unique_domains = isoform1.symmetric_difference(isoform2)
        if len(unique_domains) > 0:
            return "Domain switch"
        else: 
            return "Same status"
    except KeyError:
        # Both isoforms don't have clans, but both are translated
        
        df_domains = df.dropna(subset=['target_name'], axis=0)
        if len(df_domains) == 0:
            return 'Same status'
        elif 'isoform1' not in df_domains.index:
            return 'No domain --> Domain'
        elif 'isoform2' not in df_domains.index:
            return 'Domain --> No domain'
        
        isoform1 = get_col_set(df_domains, 'isoform1', 'target_name')
        isoform2 = get_col_set(df_domains, 'isoform2', 'target_name')
        unique_domains = isoform1.symmetric_difference(isoform2)
        if len(unique_domains) > 0:
            return "Domain switch"
        else: 
            return "Same status"

        
for i, (name, df) in zip(range(10), hmmscan_clans_isoform.groupby('event_name')):
    print '\n', df[['target_name', 'translation', 'clan_id']]
    print '\t--->', get_domain_effects(df)


         target_name  \
isoform                
isoform2         NaN   

                                                                                        translation  \
isoform                                                                                               
isoform2  HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIPVQQAWSPHSTGPTGGSSAETETDSFSLPEEYFTPAPSPGDQSS   

         clan_id  
isoform           
isoform2     NaN  
	---> No annotated reading frame --> Protein +/- domain

            target_name  \
isoform                   
isoform2  Aminotran_1_2   

                                                                                                  translation  \
isoform                                                                                                         
isoform2  NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSAYQGFASGNLERDAWAIRYFVSEGFEFFCAQSFSKNFGLYNERVGNLTVVG...   

                clan_id  
isoform                  
isoform2  PLP_aminotran  
	-

In [31]:
%time domain_effects = hmmscan_clans_isoform.groupby('event_name').apply(get_domain_effects)

CPU times: user 12.4 s, sys: 1e+03 µs, total: 12.4 s
Wall time: 12.4 s


In [32]:
domain_effects.head()

event_name
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-    No annotated reading frame --> Protein +/- domain
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-    No annotated reading frame --> Protein +/- domain
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+    No annotated reading frame --> Protein +/- domain
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-    No annotated reading frame --> Protein +/- domain
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+                                          Same status
dtype: object

In [33]:
domain_effects.name = 'isoform_domain_effects'

In [51]:
domain_effects.head()

event_name
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-    No annotated reading frame --> Protein +/- domain
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-    No annotated reading frame --> Protein +/- domain
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+    No annotated reading frame --> Protein +/- domain
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-    No annotated reading frame --> Protein +/- domain
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+                                          Same status
Name: isoform_domain_effects, dtype: object

In [52]:
domain_effects.to_csv('{}/isoform_domain_effects.csv'.format(alternative_feature_folder), header=True)

In [35]:
domain_effects.groupby(domain_effects).size()

isoform_domain_effects
Domain --> No domain                                   125
Domain switch                                           40
No annotated reading frame --> Protein +/- domain    14132
No domain --> Domain                                   654
Protein +/- domain --> No annotated reading frame     4203
Same status                                           2998
dtype: int64

### Get event to Pfam Accession and Name

In [36]:
from collections import Counter

grouped = hmmscan_filtered.groupby(['event_name', 'isoform'])
event_to_pfam_accession = grouped.apply(lambda x: ','.join('{}[{}]'.format(k, v) for k, v in Counter(x['target_accession']).items()))
event_to_pfam_accession.head()

event_name                                                                                          isoform 
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-  isoform2           nan[1]
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-  isoform2    PF00155.16[1]
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+  isoform2    PF01150.12[1]
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-  isoform2    PF02628.10[1]
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+  isoform1     PF03932.9[1]
dtype: object

In [37]:
pfam_accession_tidy = hmmscan_filtered.groupby(['event_name', 'isoform']).apply(
    lambda x: pd.Series(Counter(x['target_accession'])))
pfam_accession_tidy = pfam_accession_tidy.dropna()
pfam_accession_tidy = pfam_accession_tidy.reset_index()
pfam_accession_tidy = pfam_accession_tidy.rename(columns={'level_2': 'pfam_accession', 0: 'n_domains'})
pfam_accession_tidy.head()


,event_name,isoform,pfam_accession,n_domains
0,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,isoform2,PF00155.16,1
1,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,isoform2,PF01150.12,1
2,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,isoform2,PF02628.10,1
3,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,isoform1,PF03932.9,1
4,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,isoform2,PF03932.9,1


In [38]:
pfam_accession_to_name = pd.Series(hmmscan_filtered.target_name.values, index=hmmscan_filtered.target_accession.values,
                                  name='pfam_name')
print pfam_accession_to_name.shape
pfam_accession_to_name = pfam_accession_to_name.drop_duplicates()
pfam_accession_to_name.head()
print pfam_accession_to_name.shape

(30395,)
(2824,)


In [39]:
pfam_tidy = pfam_accession_tidy.join(pfam_accession_to_name, on='pfam_accession')
print 'pfam_tidy.shape', pfam_tidy.shape
pfam_tidy.head()

pfam_tidy.shape (14016, 5)


,event_name,isoform,pfam_accession,n_domains,pfam_name
0,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,isoform2,PF00155.16,1,Aminotran_1_2
1,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,isoform2,PF01150.12,1,GDA1_CD39
2,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,isoform2,PF02628.10,1,COX15-CtaA
3,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,isoform1,PF03932.9,1,CutC
4,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,isoform2,PF03932.9,1,CutC


In [40]:
pfam_tidy['pfam_accession_no_version'] = pfam_tidy.pfam_accession.str.split('.').str[0]
pfam_tidy.head()

,event_name,isoform,pfam_accession,n_domains,pfam_name,pfam_accession_no_version
0,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,isoform2,PF00155.16,1,Aminotran_1_2,PF00155
1,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,isoform2,PF01150.12,1,GDA1_CD39,PF01150
2,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,isoform2,PF02628.10,1,COX15-CtaA,PF02628
3,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,isoform1,PF03932.9,1,CutC,PF03932
4,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,isoform2,PF03932.9,1,CutC,PF03932


In [41]:
event_to_isoform_pfam_name = pfam_tidy.groupby(['event_name', 'isoform']).apply(
    lambda x: ','.join('{}[{}]'.format(a, int(b)) for a, b in zip(x.pfam_name, x.n_domains)))
event_to_isoform_pfam_name.name = 'pfam_name'
event_to_isoform_pfam_name.head()

event_name                                                                                          isoform 
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-  isoform2    Aminotran_1_2[1]
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+  isoform2        GDA1_CD39[1]
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-  isoform2       COX15-CtaA[1]
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+  isoform1             CutC[1]
                                                                                                    isoform2             CutC[1]
Name: pfam_name, dtype: object

In [42]:
event_to_isoform_pfam_accession = pfam_tidy.groupby(['event_name', 'isoform']).apply(
    lambda x: ','.join('{}[{}]'.format(a, int(b)) for a, b in zip(x.pfam_accession, x.n_domains)))
event_to_isoform_pfam_accession.name = 'pfam_accession'
event_to_isoform_pfam_accession.head()

event_name                                                                                          isoform 
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-  isoform2    PF00155.16[1]
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+  isoform2    PF01150.12[1]
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-  isoform2    PF02628.10[1]
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+  isoform1     PF03932.9[1]
                                                                                                    isoform2     PF03932.9[1]
Name: pfam_accession, dtype: object

### Add GO annotation

In [43]:
! wget http://www.geneontology.org/external2go/pfam2go 

pfam_to_go = pd.read_table('pfam2go', sep=' > ', skiprows=6, header=None, names=['pfam', 'go'])
# pfam_to_go.head()
pfam_to_go[['pfam_accession', 'pfam_id']] = pfam_to_go.pfam.str.split(' ').apply(pd.Series)
# pfam_to_go.head()
pfam_to_go[['go_name', 'go_id']] = pfam_to_go.go.str.split(' ; ').apply(pd.Series)
# pfam_to_go.head()
pfam_to_go['pfam_accession_only'] = pfam_to_go.pfam_accession.str.split(':').str[1]
# pfam_to_go.head()
pfam_to_go = pfam_to_go.set_index('pfam_accession_only')
# pfam_to_go.head()
pfam_to_go['go_name_only'] = pfam_to_go.go_name.str.split(':').str[1]
pfam_to_go.head()

--2016-04-14 09:49:11--  http://www.geneontology.org/external2go/pfam2go
Resolving www.geneontology.org... 52.27.86.54
Connecting to www.geneontology.org|52.27.86.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 713246 (697K) [text/plain]
Saving to: `pfam2go.1'

100%[======================================>] 713,246     2.72M/s   in 0.3s    

2016-04-14 09:49:11 (2.72 MB/s) - `pfam2go.1' saved [713246/713246]



/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


,pfam,go,pfam_accession,pfam_id,go_name,go_id,go_name_only
pfam_accession_only,,,,,,,
PF00001,Pfam:PF00001 7tm_1,GO:G-protein coupled receptor activity ; GO:0004930,Pfam:PF00001,7tm_1,GO:G-protein coupled receptor activity,GO:0004930,G-protein coupled receptor activity
PF00001,Pfam:PF00001 7tm_1,GO:G-protein coupled receptor signaling pathway ; GO:0007186,Pfam:PF00001,7tm_1,GO:G-protein coupled receptor signaling pathway,GO:0007186,G-protein coupled receptor signaling pathway
PF00001,Pfam:PF00001 7tm_1,GO:integral component of membrane ; GO:0016021,Pfam:PF00001,7tm_1,GO:integral component of membrane,GO:0016021,integral component of membrane
PF00002,Pfam:PF00002 7tm_2,GO:G-protein coupled receptor activity ; GO:0004930,Pfam:PF00002,7tm_2,GO:G-protein coupled receptor activity,GO:0004930,G-protein coupled receptor activity
PF00002,Pfam:PF00002 7tm_2,GO:G-protein coupled receptor signaling pathway ; GO:0007186,Pfam:PF00002,7tm_2,GO:G-protein coupled receptor signaling pathway,GO:0007186,G-protein coupled receptor signaling pathway


In [44]:
join_cols = pfam_to_go.columns.difference(pfam_tidy.columns)
pfam_tidy_go = pfam_tidy.join(pfam_to_go[join_cols], on='pfam_accession_no_version')
pfam_tidy_go.head()

,event_name,isoform,pfam_accession,n_domains,pfam_name,pfam_accession_no_version,go,go_id,go_name,go_name_only,pfam,pfam_id
0,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,isoform2,PF00155.16,1,Aminotran_1_2,PF00155,GO:pyridoxal phosphate binding ; GO:0030170,GO:0030170,GO:pyridoxal phosphate binding,pyridoxal phosphate binding,Pfam:PF00155 Aminotran_1_2,Aminotran_1_2
0,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,isoform2,PF00155.16,1,Aminotran_1_2,PF00155,GO:biosynthetic process ; GO:0009058,GO:0009058,GO:biosynthetic process,biosynthetic process,Pfam:PF00155 Aminotran_1_2,Aminotran_1_2
1,exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,isoform2,PF01150.12,1,GDA1_CD39,PF01150,GO:hydrolase activity ; GO:0016787,GO:0016787,GO:hydrolase activity,hydrolase activity,Pfam:PF01150 GDA1_CD39,GDA1_CD39
2,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,isoform2,PF02628.10,1,COX15-CtaA,PF02628,"GO:oxidoreductase activity, acting on the CH-CH group of donors ; GO:0016627",GO:0016627,"GO:oxidoreductase activity, acting on the CH-CH group of donors","oxidoreductase activity, acting on the CH-CH group of donors",Pfam:PF02628 COX15-CtaA,COX15-CtaA
2,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,isoform2,PF02628.10,1,COX15-CtaA,PF02628,GO:heme a biosynthetic process ; GO:0006784,GO:0006784,GO:heme a biosynthetic process,heme a biosynthetic process,Pfam:PF02628 COX15-CtaA,COX15-CtaA


In [45]:
event_to_isoform_pfam_go_id = pfam_tidy_go.groupby(['event_name', 'isoform']).apply(
    lambda x: ','.join('{}[{}]'.format(a, int(b)) for a, b in zip(x.go_id, x.n_domains)))
event_to_isoform_pfam_go_id.name = 'pfam_go_id'
event_to_isoform_pfam_go_id.head()

event_name                                                                                          isoform 
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-  isoform2                                GO:0030170[1],GO:0009058[1]
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+  isoform2                                              GO:0016787[1]
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-  isoform2    GO:0016627[1],GO:0006784[1],GO:0055114[1],GO:0016021[1]
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+  isoform1                                GO:0005507[1],GO:0055070[1]
                                                                                                    isoform2                                GO:0005507[1],GO:0055070[1]
Name: pfam_go_id, dtype: object

In [46]:
event_to_isoform_pfam_go_name = pfam_tidy_go.groupby(['event_name', 'isoform']).apply(
    lambda x: ','.join('({})[{}]'.format(a, int(b)) for a, b in zip(x.go_name_only, x.n_domains)))
event_to_isoform_pfam_go_name.name = 'pfam_go_name'
event_to_isoform_pfam_go_name.head()

event_name                                                                                          isoform 
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-  isoform2                                             (pyridoxal phosphate binding)[1],(biosynthetic process)[1]
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+  isoform2                                                                                (hydrolase activity)[1]
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-  isoform2    (oxidoreductase activity, acting on the CH-CH group of donors)[1],(heme a biosynthetic process)[...
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+  isoform1                                                    (copper ion binding)[1],(copper ion homeostasis)[1]
                                           

In [49]:
event_to_isoform_series = {'pfam_go_name': event_to_isoform_pfam_go_name, 
                           'pfam_go_id': event_to_isoform_pfam_go_id,
                           'pfam_name': event_to_isoform_pfam_name, 
                           'pfam_accession': event_to_isoform_pfam_accession}

## Save the data!!

In [50]:
for name, series in event_to_isoform_series.items():
    df = series.unstack()
    df.columns = df.columns.map(lambda x: x + '_' + name)
    df.to_csv('{}/isoform_domain_{}.csv'.format(alternative_feature_folder, name))
#     print df.head()
#     splicing_feature_data = splicing_feature_data.join(df)
# splicing_feature_data.head()